In [35]:
import os
import torch
import grb.utils as utils

In [36]:
from grb.dataset import Dataset

dataset_name = 'grb-cora'
dataset = Dataset(name=dataset_name, 
                  data_dir="../../data/",
                  mode='full',
                  feat_norm='arctan')

Dataset 'grb-cora' loaded.
    Number of nodes: 2680
    Number of edges: 5148
    Number of features: 302
    Number of classes: 7
    Number of train samples: 1608
    Number of val samples: 268
    Number of test samples: 804
    Dataset mode: full
    Feature range: [-0.9406, 0.9430]


In [37]:
adj = dataset.adj
features = dataset.features
labels = dataset.labels
num_features = dataset.num_features
num_classes = dataset.num_classes
test_mask = dataset.test_mask

In [38]:
from grb.model.torch import GCN
from grb.utils.normalize import GCNAdjNorm

model_name = "gcn"
model_sur = GCN(in_features=dataset.num_features,
                out_features=dataset.num_classes,
                hidden_features=64, 
                n_layers=2,
                adj_norm_func=GCNAdjNorm,
                layer_norm=False,
                residual=False,
                dropout=0.5)
print(model_sur)

GCN(
  (layers): ModuleList(
    (0): GCNConv(
      (linear): Linear(in_features=302, out_features=64, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): GCNConv(
      (linear): Linear(in_features=64, out_features=7, bias=True)
    )
  )
)


In [39]:
save_dir = "./saved_models/{}/{}".format(dataset_name, model_name)
save_name = "model_sur.pt"
device = "cpu"
feat_norm = None
train_mode = "inductive"  # "transductive"

In [40]:
from grb.trainer.trainer import Trainer

trainer = Trainer(dataset=dataset, 
                  optimizer=torch.optim.Adam(model_sur.parameters(), lr=0.01),
                  loss=torch.nn.functional.cross_entropy,
                  lr_scheduler=False,
                  early_stop=True,
                  early_stop_patience=500,
                  feat_norm=feat_norm,
                  device=device)

In [41]:
trainer.train(model=model_sur, 
              n_epoch=2000,
              eval_every=1,
              save_after=0,
              save_dir=save_dir,
              save_name=save_name,
              train_mode=train_mode,
              verbose=False)

  0%|          | 0/2000 [00:00<?, ?it/s]

Training early stopped. Best validation score: 0.8470
Training runtime: 8.6569.


In [42]:
# by trainer
test_score = trainer.evaluate(model_sur, dataset.test_mask)
print("Test score of surrogate model: {:.4f}".format(test_score))

Test score of surrogate model: 0.8284


In [44]:
from grb.attack.injection import TDGIA

tdgia = TDGIA(lr=0.01,
               n_epoch=1000,
               n_inject_max=100,
               n_edge_max=200,
               feat_lim_min=-1,
               feat_lim_max=1,
               device=device)

In [45]:
# Apply attack
rst = tdgia.attack(model=model,
                   adj=dataset.adj,
                   features=dataset.features,
                   target_mask=dataset.test_mask)

Attacking: Sequential inject 20/100 nodes


AttributeError: row not found